In [24]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [25]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("my-new-experiment ")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02experiment_tracking/mlruns/1', creation_time=1762560669311, experiment_id='1', last_update_time=1762560669311, lifecycle_stage='active', name='my-new-experiment ', tags={}>

In [26]:
pip install pyarrow


Note: you may need to restart the kernel to use updated packages.


In [27]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [28]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [30]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)    
    # convert duration to minutes
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["duration"] = df["duration"].dt.total_seconds() / 60

    # filter trips
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # define features
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    # convert categorical to string
    df[categorical] = df[categorical].astype(str)

    return df

In [31]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val   = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')


In [32]:
len(df_train) , len(df_val)

(73908, 61921)

In [33]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]

In [34]:
df_train

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.50,...,0.00,0.00,None,0.3,6.80,2.0,1.0,0.00,3.933333,43_151
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.00,...,2.81,0.00,None,0.3,16.86,1.0,1.0,2.75,8.750000,166_239
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.00,...,1.00,0.00,None,0.3,8.30,1.0,1.0,0.00,5.966667,41_42
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.00,...,0.00,0.00,None,0.3,9.30,2.0,1.0,0.00,7.083333,168_75
7,2,2021-01-01 00:26:31,2021-01-01 00:28:50,N,1.0,75,75,6.0,0.45,3.50,...,0.96,0.00,None,0.3,5.76,1.0,1.0,0.00,2.316667,75_75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,2,2021-01-31 21:38:00,2021-01-31 22:16:00,None,NaN,81,90,NaN,17.63,56.23,...,0.00,6.12,None,0.3,65.40,NaN,NaN,NaN,38.000000,81_90
76514,2,2021-01-31 22:43:00,2021-01-31 23:21:00,None,NaN,35,213,NaN,18.36,46.66,...,12.20,6.12,None,0.3,65.28,NaN,NaN,NaN,38.000000,35_213
76515,2,2021-01-31 22:16:00,2021-01-31 22:27:00,None,NaN,74,69,NaN,2.50,18.95,...,0.00,0.00,None,0.3,22.00,NaN,NaN,NaN,11.000000,74_69
76516,2,2021-01-31 23:10:00,2021-01-31 23:37:00,None,NaN,168,215,NaN,14.48,48.87,...,0.00,6.12,None,0.3,58.04,NaN,NaN,NaN,27.000000,168_215


In [35]:

categorical = ["PU_DO"]
numerical = ['trip_distance']

dv = DictVectorizer()

    
train_dic =df_train[categorical + numerical].to_dict(orient = 'records')
X_train = dv.fit_transform(train_dic)


val_dics= df_val[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dics)


In [36]:
target = 'duration'
y_train = df_train[target].values
y_val= df_val[target].values

In [37]:
with mlflow.start_run():

    mlflow.set_tag("developer", "omar")
    # mlflow.log_param("model_type", "linear_regression")


    lr = LinearRegression()
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(rmse)

    mlflow.log_metric("rmse" , rmse)




15.313541411272809


In [38]:
with open("/workspaces/modelslin_reg.bin" , "wb") as f_out :
    pickle.dump((dv, lr) , f_out)



In [39]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)


    with open("/workspaces/modelslin_reg.bin" , "wb") as f_out :
        pickle.dump((dv, lr) , f_out)


    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/vscode/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:53:28] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.47479
[1]	validation-rmse:10.83044
[2]	validation-rmse:10.26938
[3]	validation-rmse:9.78459
[4]	validation-rmse:9.36523
[5]	validation-rmse:9.00482
[6]	validation-rmse:8.69536
[7]	validation-rmse:8.43267
[8]	validation-rmse:8.20719
[9]	validation-rmse:8.01787
[10]	validation-rmse:7.85560
[11]	validation-rmse:7.72041
[12]	validation-rmse:7.60414
[13]	validation-rmse:7.50660
[14]	validation-rmse:7.42441
[15]	validation-rmse:7.35409
[16]	validation-rmse:7.29595
[17]	validation-rmse:7.24579
[18]	validation-rmse:7.20405
[19]	validation-rmse:7.16972
[20]	validation-rmse:7.14017
[21]	validation-rmse:7.11720
[22]	validation-rmse:7.09582
[23]	validation-rmse:7.07877
[24]	validation-rmse:7.06314
[25]	validation-rmse:7.05163
[26]	validation-rmse:7.04147
[27]	validation-rmse:7.03227
[28]	validation-rmse:7.02477
[29]	validation-rmse:7.01872
[30]	validation-rmse:7.01310
[31]	validation-rmse:7.00856
[32]	validation-rmse:7.00360
[33]	validation-rmse:7.00002
[34]	validation-rmse:

2025/11/17 01:54:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/vscode/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:54:16] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/11/17 01:54:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


with open("/workspaces/modelslin_reg.bin" , "wb") as f_out :
    pickle.dump((dv, lr) , f_out)

    فتحنا ملف اسمه modelslin_reg.bin في المسار /workspaces/

حفظنا داخله:

dv → الـ DictVectorizer

lr → موديل LinearRegression

سوّينا نسخة خاصة بنا من الموديل، وحفظناها في ملف على الجهاز.

هذه الطريقة ما لها علاقة بـ MLflow، فقط حفظ عادي ببايثون.


mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
قلنا لـ MLflow:

"خذ موديل XGBoost (booster) واحفظه داخل الـ run الحالي."

MLflow يحفظه في مجلد خاص داخل:
mlruns/<experiment_id>/<run_id>/artifacts/models_mlflow/


In [1]:
model_uri = "runs:/e263d99042754931927e90b1e1b089ed/models_mlflow"

model = mlflow.pyfunc.load_model(model_uri)



NameError: name 'mlflow' is not defined

model_uri = "runs:/e263d99042754931927e90b1e1b089ed/models_mlflow"

model = mlflow.pyfunc.load_model(model_uri)

يا MLflow، الموديل اللي أبي أحمّله موجود داخل هذا الـ Run وتحت هذا الـ artifact_path.

شرح تفاصيل الباث:

runs:/ → يعني ابحث داخل مجلد الـ mlruns

e263d99042754931927e90b1e1b089ed → هذا Run ID

models_mlflow → هذا اسم المجلد اللي بداخله الموديل (اللي استخدمته في log_model)

إذن هذا السطر يحدد "عنوان" مكان الموديل داخل MLflow.

هذا السطر يقوم بـ:

تحميل الموديل من MLflow كـ PyFuncModel جاهز للاستخدام.

بعد هذا السطر:

عندك موديل كامل جاهز

In [47]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/mlops-zoomcamp/02experiment_tracking/mlruns/1/models/m-c709d02b03814e11b7b489649e7e7a33/artifacts
  flavor: mlflow.xgboost
  run_id: e263d99042754931927e90b1e1b089ed

In [ ]:
xgboost = mlflow.xgboost.load_model(model_uri)


موديل من MLflow على شكل XGBoost Booster الأصلي.

يعني بدلاً من أن تحصل على موديل عام (PyFunc)،
تحصل هنا على الموديل الحقيقي الخاص بـ XGBoost.

In [49]:
xgboost

In [51]:
y_pred = xgboost.predict(valid)

In [52]:
y_pred[0:5]

array([18.185852 ,  7.1750865, 21.252134 , 24.262245 , 11.1397705],
      dtype=float32)